In [27]:
import torch
from transformers import pipeline

In [28]:
translator = pipeline(task="translation",
                      model="facebook/nllb-200-distilled-600M",
                      torch_dtype=torch.bfloat16) 

In [29]:
text = """\
My puppy is adorable, \
Your kitten is cute.
Her panda is friendly.
His llama is thoughtful. \
We all have nice pets!"""

In [30]:
text_translated = translator(text,
                             src_lang="eng_Latn",
                             tgt_lang="fra_Latn")

In [31]:
text_translated

[{'translation_text': 'Mon chiot est adorable, ton chaton est mignon, son panda est ami, sa lamme est attentive, nous avons tous de beaux animaux de compagnie.'}]

In [32]:
summarizer = pipeline(task="summarization",
                      model="facebook/bart-large-cnn",
                      torch_dtype=torch.bfloat16)

In [33]:
text = """	1) People throughout history have been distracted. There is nothing uniquely distracting about modern technology. We are suffering from a crisis of attention because we don't allow ourselves a mental break. In particular, we no longer daydream, which is a beneficial type of spontaneous thought.
We’re bombarded by the idea that at the root of our attention issues lies a single powerful culprit: modern technology. If we truly want to focus, it seems, we need to turn off all our devices, quit social media, and retreat into the woods for a digital detox.
 Here’s my resistance to that idea. At an elemental level, this particular era is no different than any other. there has always been a “crisis of attention.” Historically, people have turned to meditation (and other forms of contemplative(thoughtful, pensive) practice) to deal with feelings of being overwhelmed and scattered in focus, and to refocus and reflect on priorities , our inner values, intentions, purpose. This can certainly be a spiritual process, if that’s how you define it. But we’re discovering that mindfulness impacts the attention system and how it copes with the distractions that surround us and those that are generated internally. In part, that’s what meditation practitioners have always been pursuing. Think about life long ago: people in ancient India or medieval Europe didn’t have smartphones and Facebook, but they were still suffering in their own minds. They still turned to any number of practices for relief. They still described the same challenge: I’m not fully present for my life.
   A crisis of attention can happen anytime you don’t allow yourself a break. when you don’t allow your mind to rest without having any task-at-hand. Remember our distinction between mind-wandering (having off-task thoughts during a task) and daydreaming (task-free spontaneous thought and opportunity for conscious reflection, creativity, and the like)? Well, one problem today is that we are always engaged in something. With these digital tools at our fingertips, we have constant access to all these forms of communication, content, and interaction, and we don’t tend to gravitate toward letting our thoughts meander(bend), unconstrained. Of the two types of spontaneous thought we discussed earlier, it’s the beneficial type, the daydreaming, that we barely get at all. 
  We all do it. I catch myself all the time, going from one type of mental engagement to the next. I call it hype tasking. Like surfing hyperlinks online (clicking from link to link as they grab your attention), we go from one task to the next and the next. You are probably doing it right now. We are “all task and no downtime.” And we’re asking an enormous amount from our attention systems. Your attentional capacity is not less than someone’s from hundreds of years ago. It’s only that right now, you’re using your attention in a particular focused way, all the time.
  We’re taxing(exacting, demanding) our focused attention to the max. Hype tasking is hyper-taxing! Even something you might think of as relaxing (scrolling through Instagram, for example, or reading an article someone shared) is more engagement. It’s another task. Checking your notifications may seem “fun,” but it’s work for your attention. Task: check to see who posted what in response to my post. Task: check how many likes I got. Task: check who shared my funny meme. Your attention was focused on task after task after task, with no attentional downtime, not a moment for the mind to roam free.(travel aimlessly)
  It’s not always realistic to unplug. We can’t just turn off our phones and pause our email. We cannot create a distraction-free world. The issue is not the existence of this technology; rather, it’s how we’re using it: we are not allowing our minds to pay attention differently. And this is where mindfulness comes in, as a way to steady your flashlight so you don’t end up swinging it around at any and all possible distractions, digital or not.
"""

In [34]:
summary = summarizer(text,
                     min_length=50,
                     max_length=250)

In [35]:
summary

[{'summary_text': "We are suffering from a crisis of attention because we don't allow ourselves a mental break. In particular, we no longer daydream, which is a beneficial type of spontaneous thought. We can’t just turn off our phones and pause our email. We cannot create a distraction-free world."}]

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()
from transformers import pipeline
sam_pipe = pipeline("mask-generation", "Zigeng/SlimSAM-uniform-77")
from PIL import Image
raw_image = Image.open('C:/Users/Josep/OneDrive/Desktop/Edit Materials/heaven (7).jpg')
raw_image = raw_image.resize((720, 375))
output = sam_pipe(raw_image)
from helper import show_pipe_masks_on_image
show_pipe_masks_on_image(raw_image, output)
from transformers import SamModel, SamProcessor
model = SamModel.from_pretrained("./models/Zigeng/SlimSAM-uniform-77")
processor = SamProcessor.from_pretrained("./models/Zigeng/SlimSAM-uniform-77")
raw_image = raw_image.resize((720, 375))
input_points = [[[1600, 700]]]
inputs = processor(raw_image, input_points=input_points, return_tensors="pt")
import torch
with torch.no_grad():
    outputs = model(**inputs)
predicted_masks = processor.image_processor.post_process_masks(
    outputs.pred_masks, inputs["original_sizes"], inputs["reshaped_input_sizes"]
)
len(predicted_masks)
predicted_mask = predicted_masks[0]
predicted_mask.shape
outputs.iou_scores
from helper import show_mask_on_image
for i in range(3):
    show_mask_on_image(raw_image, predicted_mask[:, i])
depth_estimator = pipeline(task="depth-estimation", model="./models/Intel/dpt-hybrid-midas")
raw_image = Image.open('gradio_tamagochi_vienna.png')
raw_image = raw_image.resize((806, 621))
output = depth_estimator(raw_image)
output
output["predicted_depth"].shape
output["predicted_depth"].unsqueeze(1).shape
prediction = torch.nn.functional.interpolate(
    output["predicted_depth"].unsqueeze(1), size=raw_image.size[::-1], mode="bicubic", align_corners=False,
)
prediction.shape
raw_image.size[::-1]
prediction
import numpy as np
output = prediction.squeeze().numpy()
formatted = (output * 255 / np.max(output)).astype("uint8")
depth = Image.fromarray(formatted)
depth
import os
import gradio as gr
from transformers import pipeline
def launch(input_image):
    out = depth_estimator(input_image)

    # resize the prediction
    prediction = torch.nn.functional.interpolate(
        out["predicted_depth"].unsqueeze(1),
        size=input_image.size[::-1],
        mode="bicubic",
        align_corners=False,
    )

    # normalize the prediction
    output = prediction.squeeze().numpy()
    formatted = (output * 255 / np.max(output)).astype("uint8")
    depth = Image.fromarray(formatted)
    return depth

iface = gr.Interface(launch, 
                     inputs=gr.Image(type='pil'), 
                     outputs=gr.Image(type='pil'))
iface.launch(share=True, server_port=int(os.environ['PORT1']))
iface.close()